In [51]:
import findspark
findspark.init()

In [52]:
#pip install Faker 

In [53]:
import pyspark
from pyspark.sql import SparkSession
spark = SparkSession.builder.getOrCreate()
from pyspark.sql.functions import col,when,StringType,rand
from faker import Faker
faker = Faker()
import numpy as np
import pandas as pd


In [54]:
# conf = spark.sparkContext._conf.getAll()
# conf

In [55]:
spark.sparkContext.stop()

In [56]:
spark = SparkSession.builder.master("local[1]").appName("Spark_MLlib").getOrCreate()

In [57]:
sc= spark.sparkContext

In [9]:
#created a function to generate random data for none numeric columns
def fake_airlinedata(n,seed=None):
    output=[
        {"Flight_No":"MU-" +str(faker.pyint()),
   
         "Origin": np.random.choice(["BWI","PHL","BHM","LHD","UNK","SAN","LEX"]),
         "Destination":np.random.choice(["JFK","MIA","LFT","SDF","BOS","DTW","LAX"])
        }
        for x in range(n)         
    ]
    return output

In [10]:
# put data into dataframe
df=pd.DataFrame(fake_airlinedata(n=100,seed=0))

In [11]:
# another function for random time for scheduled arrival time and actual time columns
def flight_time(n,seed=45):
    ArrivalTime=[]
    ActualTime=[]
    ArrivalTime_min=[]
    ActualTime_min= []
    DelayTime=[]
    for _ in range(n):
        Arrivalhour= faker.date_time().hour
        Arrivalminute= faker.date_time().minute
        Actualhour= faker.date_time().hour
        Actualminute= faker.date_time().minute
        ArrivalTime.append("%s:%s" % (Arrivalhour, Arrivalminute))
        ArrivalTime_min.append(Arrivalhour*60 + Arrivalminute)
        if Actualhour > Arrivalhour:
            ActualTime.append("%s:%s" % (Actualhour,Actualminute))
            ActualTime_min.append(Actualhour*60 + Actualminute)
            DelayTime.append((Actualhour*60 + Actualminute)-(Arrivalhour*60 + Arrivalminute))
        else:
            ActualTime.append("%s:%s" % (Arrivalhour, Arrivalminute))
            ActualTime_min.append(Arrivalhour*60 + Arrivalminute)
            DelayTime.append(0)


    return ArrivalTime,ActualTime,ArrivalTime_min, ActualTime_min,DelayTime
flightTime=flight_time(100)


In [12]:
# function to generate random arrival date for arrival date column
def arrival_date(n,seed=22):
    ArrivalDate=[]
    for _ in range(n):
        ArrivalDate.append(faker.date_this_decade())
    return ArrivalDate
arrivalDate = arrival_date(100)

In [13]:
# function for actual date column
def actual_date(n,seed=23):
   
    ActualDate=arrivalDate
    return ActualDate
actualDate = actual_date(100)

In [14]:
# add all columns into the dataframe
df["ArrivalDate"]=arrivalDate
df["ArrivalTime"]=flightTime[0]
df["ActualDate"]= actualDate
df["ActualTime"]=flightTime[1]
df["Arival_min"]=flightTime[2]
df["Actual_min"]=flightTime[3]
df["DelayTime"]=flightTime[4]
df = df.astype({"Arival_min": float, "Actual_min": float,"DelayTime":float,"Flight_No":str,"Destination":str,"Origin":str})

In [45]:
df.head()

,Flight_No,Origin,Destination,ArrivalDate,ArrivalTime,ActualDate,ActualTime,Arival_min,Actual_min,DelayTime
0,MU-5991,LEX,LAX,2021-03-15,8:22,2021-03-15,13:36,502.0,816.0,314.0
1,MU-9941,LHD,DTW,2020-02-19,20:39,2020-02-19,20:39,1239.0,1239.0,0.0
2,MU-706,LHD,MIA,2020-04-06,17:27,2020-04-06,17:27,1047.0,1047.0,0.0
3,MU-1266,PHL,LAX,2021-03-11,4:28,2021-03-11,10:31,268.0,631.0,363.0
4,MU-1614,PHL,LAX,2021-04-05,2:3,2021-04-05,8:16,123.0,496.0,373.0


In [16]:
df.describe()

,Arival_min,Actual_min,DelayTime
count,100.000000,100.000000,100.000000
mean,757.610000,973.920000,216.310000
std,419.358382,321.909664,322.690229
min,8.000000,159.000000,0.000000
25%,401.250000,770.250000,0.000000
50%,785.000000,1045.500000,0.000000
75%,1160.250000,1230.750000,377.250000
max,1401.000000,1425.000000,1350.000000


In [17]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 10 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   Flight_No    100 non-null    object 
 1   Origin       100 non-null    object 
 2   Destination  100 non-null    object 
 3   ArrivalDate  100 non-null    object 
 4   ArrivalTime  100 non-null    object 
 5   ActualDate   100 non-null    object 
 6   ActualTime   100 non-null    object 
 7   Arival_min   100 non-null    float64
 8   Actual_min   100 non-null    float64
 9   DelayTime    100 non-null    float64
dtypes: float64(3), object(7)
memory usage: 7.9+ KB


In [18]:
# change the pandas dataframe into spark dataframe
spark_df = spark.createDataFrame(df)
spark_df.printSchema()



root
 |-- Flight_No: string (nullable = true)
 |-- Origin: string (nullable = true)
 |-- Destination: string (nullable = true)
 |-- ArrivalDate: date (nullable = true)
 |-- ArrivalTime: string (nullable = true)
 |-- ActualDate: date (nullable = true)
 |-- ActualTime: string (nullable = true)
 |-- Arival_min: double (nullable = true)
 |-- Actual_min: double (nullable = true)
 |-- DelayTime: double (nullable = true)



In [21]:
spark_df.show(5,truncate=False)

+---------+------+-----------+-----------+-----------+----------+----------+----------+----------+---------+
|Flight_No|Origin|Destination|ArrivalDate|ArrivalTime|ActualDate|ActualTime|Arival_min|Actual_min|DelayTime|
+---------+------+-----------+-----------+-----------+----------+----------+----------+----------+---------+
|MU-5991  |LEX   |LAX        |2021-03-15 |8:22       |2021-03-15|13:36     |502.0     |816.0     |314.0    |
|MU-9941  |LHD   |DTW        |2020-02-19 |20:39      |2020-02-19|20:39     |1239.0    |1239.0    |0.0      |
|MU-706   |LHD   |MIA        |2020-04-06 |17:27      |2020-04-06|17:27     |1047.0    |1047.0    |0.0      |
|MU-1266  |PHL   |LAX        |2021-03-11 |4:28       |2021-03-11|10:31     |268.0     |631.0     |363.0    |
|MU-1614  |PHL   |LAX        |2021-04-05 |2:3        |2021-04-05|8:16      |123.0     |496.0     |373.0    |
+---------+------+-----------+-----------+-----------+----------+----------+----------+----------+---------+
only showing top 5 

## Prediction of Flight Delay based on Arival time(Schedule Time)

In [20]:
# transform the feature(arrival time) using FeatureHassher
from pyspark.ml.feature import FeatureHasher
hasher = FeatureHasher(inputCols=["Arival_min"],
                       outputCol="features")

featurized = hasher.transform(spark_df)
featurized.show(5,truncate=False)

+---------+------+-----------+-----------+-----------+----------+----------+----------+----------+---------+--------------------------+
|Flight_No|Origin|Destination|ArrivalDate|ArrivalTime|ActualDate|ActualTime|Arival_min|Actual_min|DelayTime|features                  |
+---------+------+-----------+-----------+-----------+----------+----------+----------+----------+---------+--------------------------+
|MU-5991  |LEX   |LAX        |2021-03-15 |8:22       |2021-03-15|13:36     |502.0     |816.0     |314.0    |(262144,[216051],[502.0]) |
|MU-9941  |LHD   |DTW        |2020-02-19 |20:39      |2020-02-19|20:39     |1239.0    |1239.0    |0.0      |(262144,[216051],[1239.0])|
|MU-706   |LHD   |MIA        |2020-04-06 |17:27      |2020-04-06|17:27     |1047.0    |1047.0    |0.0      |(262144,[216051],[1047.0])|
|MU-1266  |PHL   |LAX        |2021-03-11 |4:28       |2021-03-11|10:31     |268.0     |631.0     |363.0    |(262144,[216051],[268.0]) |
|MU-1614  |PHL   |LAX        |2021-04-05 |2:3   

In [22]:
# trnaform the data by using standardScaler
from pyspark.ml.feature import StandardScaler

scaler = StandardScaler(inputCol= "features", outputCol="scaledFeatures",withStd=True, withMean=False)

# Compute summary statistics by fitting the StandardScaler
scalerModel = scaler.fit(featurized)

# Normalize each feature to have unit standard deviation.
scaledData = scalerModel.transform(featurized)
scaledData.show(5)

+---------+------+-----------+-----------+-----------+----------+----------+----------+----------+---------+--------------------+--------------------+
|Flight_No|Origin|Destination|ArrivalDate|ArrivalTime|ActualDate|ActualTime|Arival_min|Actual_min|DelayTime|            features|      scaledFeatures|
+---------+------+-----------+-----------+-----------+----------+----------+----------+----------+---------+--------------------+--------------------+
|  MU-5991|   LEX|        LAX| 2021-03-15|       8:22|2021-03-15|     13:36|     502.0|     816.0|    314.0|(262144,[216051],...|(262144,[216051],...|
|  MU-9941|   LHD|        DTW| 2020-02-19|      20:39|2020-02-19|     20:39|    1239.0|    1239.0|      0.0|(262144,[216051],...|(262144,[216051],...|
|   MU-706|   LHD|        MIA| 2020-04-06|      17:27|2020-04-06|     17:27|    1047.0|    1047.0|      0.0|(262144,[216051],...|(262144,[216051],...|
|  MU-1266|   PHL|        LAX| 2021-03-11|       4:28|2021-03-11|     10:31|     268.0|     63

In [39]:
#split the data into train and test
splits = scaledData.randomSplit([0.7, 0.3])
train_df = splits[0]
test_df = splits[1]

In [40]:
# estimator LinearRegression, get -0.56 for correlation cofficient which shows us that the two features have negative relationship.
from pyspark.ml.regression import LinearRegression
lr = LinearRegression(featuresCol = 'features', labelCol='DelayTime', maxIter=10, regParam=0.3, elasticNetParam=0.8)
lr_model = lr.fit(train_df)
print("Coefficients: " + str(lr_model.coefficients))
print("Intercept: " + str(lr_model.intercept))


Coefficients: (262144,[216051],[-0.5331734167610094])
Intercept: 638.3338931352447


In [41]:
#the model evaluation
trainingSummary = lr_model.summary
print("RMSE: %f" % trainingSummary.rootMeanSquaredError)
print("r2: %f" % trainingSummary.r2)

RMSE: 257.404371
r2: 0.452742


In [42]:
#model prediction for the test data
test_result = lr_model.evaluate(test_df)
print("Root Mean Squared Error (RMSE) on test data = %g" % test_result.rootMeanSquaredError)

Root Mean Squared Error (RMSE) on test data = 211.622


In [43]:
print("numIterations: %d" % trainingSummary.totalIterations)
print("objectiveHistory: %s" % str(trainingSummary.objectiveHistory))
trainingSummary.residuals.show(5)

numIterations: 3
objectiveHistory: [0.49285714285714294, 0.42878315938634637, 0.2702187374135515]
+-------------------+
|          residuals|
+-------------------+
| -21.98542335951788|
|  611.4598701184347|
|  88.81717239362416|
|-15.054168941624766|
| 116.88989734604672|
+-------------------+
only showing top 5 rows



In [44]:
predictions = lr_model.transform(test_df)
predictions.select("prediction","DelayTime","features").show(5)

+------------------+---------+--------------------+
|        prediction|DelayTime|            features|
+------------------+---------+--------------------+
| 572.7535628736405|    373.0|(262144,[216051],...|
|-94.77955491114324|      0.0|(262144,[216051],...|
|269.37788873662623|      0.0|(262144,[216051],...|
|235.25479006392163|      0.0|(262144,[216051],...|
| 404.8039365939226|    573.0|(262144,[216051],...|
+------------------+---------+--------------------+
only showing top 5 rows

